# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

from dotenv import load_dotenv
import os

# Load the .env file from the specified directory
load_dotenv(dotenv_path='../../01_materials/labs/.env')

# Check if the environment variables are loaded correctly
price_data_path = os.getenv('PRICE_DATA')
if price_data_path is None:
    raise ValueError("PRICE_DATA environment variable is not set.")
print(f'PRICE_DATA path: {price_data_path}')



PRICE_DATA path: ../../05_src/data/prices/


In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

parquet_files = glob(os.path.join(price_data_path, '**', '*.parquet'), recursive=True)
print(f'Found {len(parquet_files)} parquet files.')



Found 22484 parquet files.


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.

import dask.dataframe as dd
import os
import glob

# Define the path to your parquet files
price_data_path = "../../05_src/data/prices/"

# List all parquet files recursively
def list_parquet_files(directory):
    parquet_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".parquet"):
                parquet_files.append(os.path.join(root, file))
    return parquet_files

# Load all parquet files into a Dask DataFrame
parquet_files = list_parquet_files(price_data_path)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

# Define meta for expected output columns and types
meta = {
    'Close_lag_1': 'f8',          # Float type for Close lag
    'Adj_Close_lag_1': 'f8',      # Float type for Adjusted Close lag
    'returns': 'f8',              # Float type for returns
    'hi_lo_range': 'f8'           # Float type for high-low range
}

# Group by 'ticker' and calculate lags, returns, and range
dd_feat = (
    dd_px.groupby('ticker', group_keys=False)
    .apply(lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1),
        returns = (x['Adj Close'] / x['Adj Close'].shift(1)) - 1,
        hi_lo_range = x['High'] - x['Low']
    ), meta=meta)
)

# Print the first few rows of dd_feat for verification
print(dd_feat.head())











Empty DataFrame
Columns: [Close_lag_1, Adj_Close_lag_1, returns, hi_lo_range]
Index: []


/Users/mishavitvinov/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/core.py:8391: UserWarning: Insufficient elements for `head`. 5 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [5]:
print(dd_feat.columns)


Index(['Close_lag_1', 'Adj_Close_lag_1', 'returns', 'hi_lo_range'], dtype='object')


In [6]:
# Write your code below.


# Import necessary libraries
import os
import glob
import dask.dataframe as dd
import pandas as pd

# Define the path to your parquet files
price_data_path = "../../05_src/data/prices/"

# List all parquet files recursively
def list_parquet_files(directory):
    parquet_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".parquet"):
                parquet_files.append(os.path.join(root, file))
    return parquet_files

# Load all parquet files into a Dask DataFrame
parquet_files = list_parquet_files(price_data_path)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

# Group by 'ticker' and calculate lags, returns, and range
dd_feat = (
    dd_px.groupby('ticker', group_keys=False)
    .apply(lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1),
        returns = (x['Adj Close'] / x['Adj Close'].shift(1)) - 1,
        hi_lo_range = x['High'] - x['Low']
    ))
)

# Convert Dask DataFrame to Pandas DataFrame
df_px_pd = dd_feat.compute()

# Reset index to avoid duplicate index labels
df_px_pd = df_px_pd.reset_index(drop=True)

# Check column names to verify 'ticker' column is present
print(df_px_pd.columns)

# Calculate rolling average return with window of 10 days
if 'ticker' in df_px_pd.columns:
    df_px_pd['rolling_avg_return'] = df_px_pd.groupby('ticker')['returns'].rolling(10).mean().reset_index(level=0, drop=True)
else:
    print("Error: 'ticker' column not found in DataFrame.")

# Print the first few rows of df_px_pd for verification
print(df_px_pd.head())











/var/folders/g5/lrw0qppn61vdzgfk_np6yxmr0000gn/T/ipykernel_17240/1438167890.py:28: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_px.groupby('ticker', group_keys=False)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns',
       'hi_lo_range'],
      dtype='object')
Error: 'ticker' column not found in DataFrame.
        Date       Open       High        Low      Close  Adj Close   Volume  \
0 2005-01-03  27.956232  27.956232  27.026913  27.089188  16.288546  3756753   
1 2005-01-04  27.089188  27.175411  26.768236  26.787397  16.107073  4629348   
2 2005-01-05  25.953884  26.959848  25.949095  26.638899  16.017790  5355607   
3 2005-01-06  26.638899  27.367023  26.590996  27.146669  16.323109  3806228   
4 2005-01-07  27.146669  27.146669  26.629316  26.705961  16.058121  7627486   

   sector                           subsector  year  Close_lag_1  \
0  Energy  Oil & Gas Exploration & Production  2005          NaN   
1  Energy  Oil & Gas Exploration & Production  2005    27.089188   
2  Energy  Oil & Gas Exploration & Production  2005    26.787397   
3 

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

1) Converting to Pandas was necessary in this case because Pandas provides direct support for rolling calculations like .rolling(10).mean(). Dask, on the other hand, does not directly support rolling operations that involve dependencies across partitions. Therefore, to perform the rolling average return calculation efficiently and effectively, converting to Pandas was the practical choice.

2) Performing the rolling average return calculation in Dask could have been challenging and less efficient due to Dask's limitations with rolling operations across partitions. Dask is optimized for out-of-core and parallel computation but lacks direct support for rolling calculations that require dependencies across chunks of data. Hence, while Dask is suitable for handling large datasets and parallel processing, certain complex operations, like rolling calculations, are better executed in Pandas where the data can be consolidated into memory for efficient computation.



## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.